In [1]:
from PIL import Image
import pywt
import os
import numpy as np
import pandas as pd
from sklearn import preprocessing
import sys

original_stdout = sys.stdout

In [2]:
PATH = './leedsbutterfly/images/'
FILE_NAMES = os.listdir(PATH)

In [3]:
def get_FV(image, it=5):
    LL, (LH, HL, HH) = pywt.dwt2(image, 'haar')
    for _ in range(it):
        LL, (LH, HL, HH) = pywt.dwt2(LL, 'haar')
    return LL.flatten()

In [4]:
# cuts = [6]
# for c in cuts:
#     FVs = []
#     for im in os.listdir(PATH):
#         image = Image.open(PATH + im).resize((100, 100))
#         mt_image = np.asarray(image).astype(int)
#         FVs.append(get_FV(mt_image, c).tolist())
#     cols = len(FVs[0])
#     df = pd.DataFrame(FVs, columns=[str(_) for _ in range(cols)])
#     x = df.values #returns a numpy array
#     min_max_scaler = preprocessing.MinMaxScaler()
#     x_scaled = min_max_scaler.fit_transform(x)
#     df = pd.DataFrame(x_scaled)
    
#     df.to_csv(f'./data_stratification/leedsbutterfly_{c}.csv', index=False)

In [5]:
print(np.unique([int(FILE_NAMES[_][0:3]) for _ in range(len(FILE_NAMES))]))

[ 1  2  3  4  5  6  7  8  9 10]


In [6]:
y_expected = []

for _ in range(len(FILE_NAMES)):
    arr = np.zeros(10).astype(int)
    arr[int(FILE_NAMES[_][0:3])-1] = 1
    y_expected.append(list(arr))

y_csv = pd.DataFrame(y_expected, columns=[str(_) for _ in range(10)])
y_csv.to_csv('./data_stratification/y_expected.csv', index=False)

In [7]:
from random import shuffle

In [8]:
c = 6
y_csv = pd.read_csv(f'./data_stratification/y_expected.csv')
df = pd.read_csv(f'./data_stratification/leedsbutterfly_{c}.csv')

In [9]:
unique_ids = list(np.unique([''.join(i) for i in zip(*[y_csv[label].astype(str) for label in y_csv.columns])]))

In [10]:
y_csv['label'] = [''.join(i) for i in zip(*[y_csv[label].astype(str) for label in y_csv.columns])]

In [11]:
indexes = []
for one_hot in y_csv['label'].unique():
    indexes.append(np.where(y_csv['label'] == one_hot)[0].tolist())


#### **leed_buttefly str.**

In [32]:
df_new = pd.DataFrame(columns=df.columns)

idx = 0

while True:
    past_len = len(df_new)
    for i in range(len(indexes)):
        try:
            # print(indexes[i][idx], end=" ")
            df_new.loc[len(df_new)] = df.loc[indexes[i][idx]]
        except Exception as e:
            continue
    idx += 1
    if past_len == len(df_new):
        break

df_new.to_csv('./data_stratification/leedsbutterfly_6_stratified.csv', index=False)

#### **y_expected str.**

In [12]:
idx = 0
new_y = []

while True:
    past_len = len(new_y)
    for i in range(len(indexes)):
        try:
            # print(indexes[i][idx], end=" ")
            # print(y_csv.iloc[indexes[i][idx]]['label'])   
            new_y.append(y_csv.iloc[indexes[i][idx]]['label'])
        except Exception as e:
            continue
            # print(e.__class__, e)
    idx += 1
    if past_len == len(new_y):
        break

In [14]:
with open('out.txt', 'w') as f:
    sys.stdout = f
    for ny in new_y:
        print(ny)
    sys.stdout = original_stdout
    f.close()

In [29]:
for i in range(len(new_y)):
    new_y[i] = list(new_y[i])

df_new_y = pd.DataFrame(new_y)
df_new_y.to_csv('./data_stratification/y_expected_stratified.csv', index=False)